# Logistic Regression with FedGVI

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import logging

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pathlib import Path

import math
from __future__ import division
import copy
from statistics import fmean

import torch
import torch.utils.data
import torch.nn as nn
from torch import distributions, nn, optim
from torch.distributions import MultivariateNormal
from torchvision import transforms, datasets
from scipy.stats import multivariate_normal
import scipy.stats as stats
import numpy as np
from tqdm import tqdm

import scipy.io
from sklearn.model_selection import train_test_split

from fedgvi_experiments.logistic_regression.LogRegExp import LogReg
from DSVGD.Bayesian_Logistic_Regression.Methods import Methods
from fedgvi_experiments.utils.helper_functions import Gaussian

from tueplots import bundles, fonts
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("pgf")
matplotlib.rcParams.update(fonts.icml2024_tex(family="serif"))
#import tqdm.auto as tqdm

JITTER = 1e-8

%matplotlib inline
device = 'cpu'
torch.set_default_dtype(torch.float64)
torch.set_default_device(device = device)


In [ ]:
def plot_data(x, y):
    x_vals = x[:, 0]
    y_vals = x[:, 1]
    labels = y
    
    plt.figure()
    plt.grid(visible=True)
    plt.scatter(x_vals, y_vals, c=labels)
    plt.show()

seeds = [2, 13, 21, 42, 71, 97, 151, 227, 331, 397]
cs = [(255/255,176/255,0/255),(254/255,97/255,0/255),(220/255,38/255,127/255), 
      (120/255,94/255,240/255),(100/255,143/255,255/255)]

cs2 = [(238/255,119/255,51/255), (0/255,119/255,187/255), (51/255,187/255,238/255),
       (238/255,51/255,119/255), (204/255,51/255,17/255), (0/255,153/255,136/255)]


def min_max_avg_validation(list_):
    min_ = []
    max_ = []
    avg = []
    iters = []
    for i in range(len(list_[0]["validation"])):
        iters.append(i+1)
        val = []
        val_4_i = []
        val_5_i = []
        for j in range(len(list_)):
            val.append(list_[j]["validation"][i])
        min_.append(min(val))
        max_.append(max(val))
        avg.append(fmean(val))
    return [min_, max_, avg]

## FedGVI - Covertype Data set

#### 2 Clients averaged over 10 train/test splits

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.1)
s = a.sample((100,))
run3 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=2,
                data_set="covertype",
                augment=True,#w^Tx+b if true, w^Tx otherwise
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.2},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=500,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                diff_prior_cov = None,# torch.tensor([1/s.mean()]),
                client_div = "KLD",
                client_div_param = 1.0,
                client_loss = "nll",
                client_score_fct = None,
                client_loss_param = None,
                seed=seed_)
    run3.append(ret)
    print("\n")

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
run4 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=2,
                data_set="covertype",
                augment=True,
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.6},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=500,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                diff_prior_cov = torch.tensor([1 / s.mean()]),
                client_div = "AR",
                client_div_param = 2.5,
                client_loss = "nll",
                client_score_fct = None,
                client_loss_param = None,
                seed=seed_)
    run4.append(ret)
    print("\n")

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
run5 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=2,
                data_set="covertype",
                augment=True,
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.6},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=500,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                diff_prior_cov = torch.tensor([1 / s.mean()]),
                client_div = "KLD",
                client_div_param = 0.6,
                client_loss = "gen_CE",
                client_score_fct = None,
                client_loss_param = 0.5,
                seed=seed_,
                global_div="KLD",
                global_div_param=None)
    run5.append(ret)
    print("\n")

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
run5_1 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=2,
                data_set="covertype",
                augment=True,
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.6},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=500,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                #diff_prior_cov = torch.tensor([1 / s.mean()]),
                client_div = "KLD",
                client_div_param = 0.6,
                client_loss = "density_power",
                client_score_fct = None,
                client_loss_param = 0.5,
                seed=seed_,
                global_div="KLD",
                global_div_param=None)
    run5_1.append(ret)
    print("\n")

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
run5_2 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=2,
                data_set="covertype",
                augment=True,
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.6},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=500,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                #diff_prior_cov = torch.tensor([1 / s.mean()]),
                client_div = "KLD",
                client_div_param = 0.6,
                client_loss = "gamma_mislabel",
                client_score_fct = None,
                client_loss_param = 0.75,
                seed=seed_,
                global_div="KLD",
                global_div_param=None)
    run5_2.append(ret)
    print("\n")

#### Plot 2 Client Results

In [ ]:
min3, max3, min4, max4, min5, max5 = [],[],[],[],[],[]
avg3, avg4, avg5 = [], [], []
iters = []
for i in range(10):
    iters.append(i+1)
    val_3_i = []
    val_4_i = []
    val_5_i = []
    for j in range(len(seeds)):
        val_3_i.append(run3[j]["validation"][i])
        val_4_i.append(run4[j]["validation"][i])
        val_5_i.append(run5[j]["validation"][i])
    min3.append(min(val_3_i))
    max3.append(max(val_3_i))
    min4.append(min(val_4_i))
    max4.append(max(val_4_i))
    min5.append(min(val_5_i))
    max5.append(max(val_5_i))
    avg3.append(fmean(val_3_i))
    avg4.append(fmean(val_4_i))
    avg5.append(fmean(val_5_i))

In [ ]:
with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.25), dpi = 200)
    ax.plot(iters,avg3, marker='D', label=r"PVI")
    ax.fill_between(iters, min3, max3, alpha=0.3)
    ax.plot(iters,avg4, marker='o', label=r"$D_{AR}^{(5)}$")
    ax.fill_between(iters, min4, max4, alpha=0.3)
    ax.plot(iters,avg5, marker='^', label=r"$\mathcal{L}_{GCE}^{(0.5)}$")
    ax.fill_between(iters, min5, max5, alpha=0.3)
    ax.set_xlabel(r"Server Iterations $t$")
    ax.set_ylabel(r"Classification Accuracy")
    ax.legend(loc="lower right", frameon=True)
    plt.grid(visible=True)
    plt.show()

#### 1 Client

In [ ]:
#VI
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
run6 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=1,
                data_set="covertype",
                augment=True,#w^Tx+b if true, w^Tx otherwise
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.2},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=5000,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                diff_prior_cov = torch.tensor([1/s.mean()]),
                client_div = "KLD",
                client_div_param = 0.5,
                client_loss = "nll",
                client_score_fct = None,
                client_loss_param = None,
                seed=seed_)
    run6.append(ret)

In [ ]:
#GVI
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.1)
s = a.sample((100,))
run7 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=1,
                data_set="covertype",
                augment=True,
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.2},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=5000,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                diff_prior_cov = torch.tensor([1 / s.mean()]),
                client_div = "AR",
                client_div_param = 5,
                client_loss = "nll",
                client_score_fct = None,
                client_loss_param = None,
                seed=seed_)
    run7.append(ret)

In [ ]:
#GVI_2
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.1)
s = a.sample((100,))
run8 = []
for seed_ in seeds:
    print("Round ", seeds.index(seed_) + 1)
    ret = LogReg().run(num_clients=1,
                data_set="covertype",
                augment=True,
                train_test_split = 0.2,
                contamination={"contaminate": False, "type": None, "epsilon": 0.2},
                heterogeneity=False,
                server_iterations=10,
                optim_epochs=5000,
                lr = 0.01,
                monte_carlo_samples=150,
                minibatch_size = 1024,
                client_batch_frac = 1.0,
                diff_prior_loc = None,
                diff_prior_cov = torch.tensor([1 / s.mean()]),
                client_div = "KLD",
                client_div_param = 1,
                client_loss = "gen_CE",
                client_score_fct = None,
                client_loss_param = 0.75,
                seed=seed_)
    run8.append(ret)

In [ ]:
min6, max6, min7, max7, min8, max8 = [],[],[],[],[],[]
avg6, avg7, avg8 = [], [], []
iters = []
for i in range(len(seeds)):
    iters.append(i+1)
    val_6_i = []
    val_7_i = []
    val_8_i = []
    for j in range(len(seeds)):
        val_6_i.append(run6[j]["validation"][i])
        val_7_i.append(run7[j]["validation"][i])
        val_8_i.append(run8[j]["validation"][i])
    min6.append(min(val_6_i))
    max6.append(max(val_6_i))
    min7.append(min(val_7_i))
    max7.append(max(val_7_i))
    min8.append(min(val_8_i))
    max8.append(max(val_8_i))
    avg6.append(fmean(val_6_i))
    avg7.append(fmean(val_7_i))
    avg8.append(fmean(val_8_i))

In [ ]:
with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.25), dpi = 200)
    
    ax.plot(iters,avg3, marker='D', label=r"PVI")
    ax.fill_between(iters, min3, max3, alpha=0.3)
    ax.plot(iters,avg4, marker='o', label=r"FedGVI, $D_{AR}^{(5)}$")
    ax.fill_between(iters, min4, max4, alpha=0.3)
    ax.plot(iters,avg5, marker='^', label=r"FedGVI, $\mathcal{L}_{GCE}^{(0.5)}$")
    ax.fill_between(iters, min5, max5, alpha=0.3)
    
    ax.plot(iters,avg6, marker='s', label=r"VI")
    ax.fill_between(iters, min6, max6, alpha=0.3)
    ax.plot(iters,avg7, marker='p', label=r"GVI, $D_{AR}^{(5)}$")
    ax.fill_between(iters, min7, max7, alpha=0.3)
    ax.plot(iters,avg8, marker='v', label=r"GVI, $\mathcal{L}_{GCE}^{(0.5)}$")
    ax.fill_between(iters, min8, max8, alpha=0.3)
    
    ax.set_xlabel(r"Server Iterations $t$")
    ax.set_ylabel(r"Classification Accuracy")
    ax.legend(loc="lower right", frameon=True, ncol=2)
    plt.grid(visible=True)
    fig.savefig('./figs/log_reg_fedgvi_vi.pgf', format="pgf", bbox_inches= "tight", pad_inches=0)
    plt.show()

#### ROC and AUC curves

In [ ]:
data = scipy.io.loadmat('./../data/covertype.mat')
X_input_ = data['covtype'][:, 1:]

X_input = np.column_stack((np.ones(len(X_input_)), X_input_))# If w^T X + b is desired

y_input = data['covtype'][:, 0]
y_input[y_input == 2] = 0  # ensure labels are in {0, 1} otherwise logistic loss breaks but we relabel as needed

data_set = {
    "x": torch.tensor(X_input),
    "y": torch.tensor(y_input),
}

In [ ]:
mu1 = run3[7]["q"]["loc"].detach()
sigma1 = run3[7]["q"]["var"].detach()
mu2 = run5[6]["q"]["loc"].detach()
sigma2 = run5[6]["q"]["var"].detach()
mu3 = run4[2]["q"]["loc"].detach()
sigma3 = run4[2]["q"]["var"].detach()
probabilities_run3 = []
probabilities_run5 = []
probabilities_run4 = []
for j in range(2):
    if j == 0:
        mu= mu1
        sigma=sigma1
    elif j == 1:
        mu = mu2
        sigma=sigma2
    elif j == 2:
        mu = mu3
        sigma=sigma3
    for i in tqdm (range(len(data_set["y"])), desc="Validating", colour='green'):
        assert data_set["y"][i] == 0 or data_set["y"][i] == 1, "Not {0,1} loss labels."
    
        x = data_set["x"][i]
        z = (mu @ x) / torch.sqrt(1 + np.pi * (x @ torch.diag(sigma) @ x))
        apx_sigmoid = torch.special.expit(z)
        
        if math.isnan(apx_sigmoid):
            print("NaN encountered", i, "\n", data_set["x"][i])
            break
        if j == 0:
            probabilities_run3.append(apx_sigmoid)
        elif j==1:
            probabilities_run5.append(apx_sigmoid)
        elif j==2:
            probabilities_run4.append(apx_sigmoid)

In [ ]:
from sklearn.metrics import roc_auc_score

pvi_auc = roc_auc_score(y_input, torch.tensor(probabilities_run3))
fedgvi_loss_auc = roc_auc_score(y_input, torch.tensor(probabilities_run5))
fedgvi_div_auc = roc_auc_score(y_input, torch.tensor(probabilities_run4))

"PVI AUC: ", pvi_auc, ". FedGVI with AR Div AUC: ", fedgvi_div_auc, ". FedGVI with GCE AUC: ", fedgvi_loss_auc

In [ ]:
#ROC Curve FedGVI
x_roc_fedgvi = []
y_roc_fedgvi = []
for pair in Results[1]:
    x_roc_fedgvi.append(pair[0])
    y_roc_fedgvi.append(pair[1])

with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.25), dpi = 200)
    
    ax.plot(x_roc_fedgvi,y_roc_fedgvi, marker='D', label=r"FedGVI")
    ax.plot(x_roc_pvi,y_roc_pvi, marker='+', label=r"PVI")
    
    ax.set_xlabel(r"$1-S_p$")
    ax.set_ylabel(r"$S_e$")
    ax.legend(loc="lower right", frameon=True, ncol=2)
    ax.set_xlim([0.0,1.0])
    ax.set_ylim([0.0,1.0])
    plt.grid(visible=True)
    fig.savefig('./figs/ROC_FedGVI.pgf', format="pgf", bbox_inches= "tight", pad_inches=0)
    plt.show()

## Synthetic Data set

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
s_run0 = []
seed_ = seeds[2]
ret = LogReg().run(num_clients=5,
            data_set="synthetic_4",
            augment=True,#w^Tx+b if true, w^Tx otherwise
            train_test_split = 0.0,
            contamination={"contaminate": False, "type": "adversarial1", "epsilon": 0.05},
            heterogeneity=False,
            server_iterations=20,
            optim_epochs=500,
            lr = 0.01,
            monte_carlo_samples=100,
            minibatch_size = np.inf,
            client_batch_frac = 1.0,
            diff_prior_loc = None,
            diff_prior_cov = torch.tensor([1/s.mean()]),
            client_div = "KLD",
            client_div_param = 1,
            client_loss = "nll",
            client_score_fct = None,
            client_loss_param = None,
            seed=seed_,
            return_data=True)
s_run0.append(ret)

In [ ]:
x_ = np.linspace(-3, 7, 500)
y_ = np.linspace(-3, 7, 500)

X, Y = np.meshgrid(x_,y_)

tx = s_run0[0]["train_data"]["x"]
ty = s_run0[0]["train_data"]["y"]
x_vals = tx[:, 1]
y_vals = tx[:, 2]
labels = ty

mu_s = s_run0[0]["q"]["loc"]
cov_s = s_run0[0]["q"]["var"]
num_samps = 1000
thetas = MultivariateNormal(loc=mu_s, covariance_matrix=torch.diag(cov_s)).sample((num_samps,)).numpy()
Probs0 = np.zeros_like(X)

for theta in thetas:
    trial1 = theta[1]*X + theta[2]*Y
    trial2 = theta[0] + trial1
    trial3 = -trial2
    trial4 = np.exp(trial3)
    trial5 = 1 + trial4
    trial6 = 1 / trial5
    #Probs += 1/(1+ np.exp(-(theta[0] + theta[1]*X + theta[2]*Y)))
    Probs0 = Probs0 + trial6
Probs0 = Probs0/num_samps

with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.5), dpi = 200)
    plt.grid(visible=True)
    ax.scatter(x_vals, y_vals, c=labels)
    ax.contour(X,Y,Probs0, levels=[0.2,0.5,0.8], colors="red")
    #ax.legend(loc="top left", frameon=True)
    plt.show()

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
s_run1 = []
seed_ = seeds[0]
ret = LogReg().run(num_clients=5,
            data_set="synthetic_4",
            augment=True,#w^Tx+b if true, w^Tx otherwise
            train_test_split = None,
            contamination={"contaminate": True, "type": "adversarial1", "epsilon": 0.05},
            heterogeneity=False,
            server_iterations=20,
            optim_epochs=500,
            lr = 0.01,
            monte_carlo_samples=100,
            minibatch_size = np.inf,
            client_batch_frac = 1.0,
            diff_prior_loc = None,
            diff_prior_cov = torch.tensor([1/s.mean()]),
            client_div = "KLD",
            client_div_param = 1,
            client_loss = "nll",
            client_score_fct = None,
            client_loss_param = None,
            seed=seed_,
            return_data=True)
s_run1.append(ret)

In [ ]:
x_ = np.linspace(-3, 7, 500)
y_ = np.linspace(-3, 7, 500)

X, Y = np.meshgrid(x_,y_)

tx = s_run1[0]["train_data"]["x"]
ty = s_run1[0]["train_data"]["y"]
x_vals = tx[:, 1]
y_vals = tx[:, 2]
labels = ty

mu_s = s_run1[0]["q"]["loc"]
cov_s = s_run1[0]["q"]["var"]
num_samps = 1000
thetas = MultivariateNormal(loc=mu_s, covariance_matrix=torch.diag(cov_s)).sample((num_samps,)).numpy()
Probs = np.zeros_like(X)

for theta in thetas:
    trial1 = theta[1]*X + theta[2]*Y
    trial2 = theta[0] + trial1
    trial3 = -trial2
    trial4 = np.exp(trial3)
    trial5 = 1 + trial4
    trial6 = 1 / trial5
    #Probs += 1/(1+ np.exp(-(theta[0] + theta[1]*X + theta[2]*Y)))
    Probs = Probs + trial6
Probs = Probs/num_samps

with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.5), dpi = 200)
    plt.grid(visible=True)
    ax.scatter(x_vals, y_vals, c=labels)
    ax.contour(X,Y,Probs, levels=[0.2,0.5,0.8], colors="red")
    plt.show()

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
s_run2 = []
seed_ = seeds[0]
ret = LogReg().run(num_clients=5,
            data_set="synthetic_4",
            augment=True,#w^Tx+b if true, w^Tx otherwise
            train_test_split = 0.0,
            contamination={"contaminate": True, "type": "adversarial1", "epsilon": 0.05},
            heterogeneity=False,
            server_iterations=20,
            optim_epochs=500,
            lr = 0.01,
            monte_carlo_samples=100,
            minibatch_size = np.inf,
            client_batch_frac = 1.0,
            diff_prior_loc = None,
            diff_prior_cov = torch.tensor([1/s.mean()]),
            client_div = "AR",
            client_div_param = 1.5,
            client_loss = "density_power",
            client_score_fct = None,
            client_loss_param = 0.7,
            seed=seed_,
            return_data=True)
s_run2.append(ret)

In [ ]:
x_ = np.linspace(-3, 7, 500)
y_ = np.linspace(-3, 7, 500)

X, Y = np.meshgrid(x_,y_)

tx = s_run2[0]["train_data"]["x"]
ty = s_run2[0]["train_data"]["y"]
x_vals = tx[:, 1]
y_vals = tx[:, 2]
labels = ty

mu_s2 = s_run2[0]["q"]["loc"]
cov_s2 = s_run2[0]["q"]["var"]

num_samps = 1000
thetas = MultivariateNormal(loc=mu_s2, covariance_matrix=torch.diag(cov_s2)).sample((num_samps,)).numpy()
Probs2 = np.zeros_like(X)

for theta in thetas:
    trial1 = theta[1]*X + theta[2]*Y
    trial2 = theta[0] + trial1
    trial3 = -trial2
    trial4 = np.exp(trial3)
    trial5 = 1 + trial4
    trial6 = 1 / trial5
    #Probs += 1/(1+ np.exp(-(theta[0] + theta[1]*X + theta[2]*Y)))
    Probs2 = Probs2 + trial6
Probs2 = Probs2/num_samps

with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.5), dpi = 200)
    plt.grid(visible=True)
    ax.scatter(x_vals, y_vals, c=labels)
    ax.contour(X,Y,Probs2, levels=[0.2,0.5,0.8], colors="blue")
    ax.contour(X,Y,Probs, levels=[0.2,0.5,0.8], colors="red")
    ax.contour(X,Y,Probs0, levels=[0.2,0.5,0.8], colors="black", alpha=0.3)
    plt.show()

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
s_run3 = []
seed_ = seeds[0]
ret = LogReg().run(num_clients=5,
            data_set="synthetic_4",
            augment=True,#w^Tx+b if true, w^Tx otherwise
            train_test_split = 0.0,
            contamination={"contaminate": True, "type": "adversarial1", "epsilon": 0.05},
            heterogeneity=False,
            server_iterations=20,
            optim_epochs=500,
            lr = 0.005,
            monte_carlo_samples=100,
            minibatch_size = np.inf,
            client_batch_frac = 1.0,
            diff_prior_loc = None,
            diff_prior_cov = torch.tensor([1/s.mean()]),
            client_div = "KLD",
            client_div_param = None,
            client_loss = "gamma_mislabel",
            client_score_fct = None,
            client_loss_param = 0.7,
            seed=seed_,
            return_data=True)
s_run3.append(ret)

In [ ]:
x_ = np.linspace(-3, 7, 500)
y_ = np.linspace(-3, 7, 500)

X, Y = np.meshgrid(x_,y_)

tx = s_run3[0]["train_data"]["x"]
ty = s_run3[0]["train_data"]["y"]
x_vals = tx[:, 1]
y_vals = tx[:, 2]
labels = ty

mu_s = s_run3[0]["q"]["loc"]
cov_s = s_run3[0]["q"]["var"]
num_samps = 1000
thetas = MultivariateNormal(loc=mu_s, covariance_matrix=torch.diag(cov_s)).sample((num_samps,)).numpy()
Probs3 = np.zeros_like(X)

for theta in thetas:
    trial1 = theta[1]*X + theta[2]*Y
    trial2 = theta[0] + trial1
    trial3 = -trial2
    trial4 = np.exp(trial3)
    trial5 = 1 + trial4
    trial6 = 1 / trial5
    #Probs += 1/(1+ np.exp(-(theta[0] + theta[1]*X + theta[2]*Y)))
    Probs3 = Probs3 + trial6
Probs3 = Probs3/num_samps

with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.5), dpi = 200)
    plt.grid(visible=True)
    ax.scatter(x_vals, y_vals, c=labels)
    ax.contour(X,Y,Probs, levels=[0.2,0.5,0.8], colors="red")
    ax.contour(X,Y,Probs3, levels=[0.2,0.5,0.8], colors="black")
    ax.contour(X,Y,Probs0, levels=[0.2,0.5,0.8], colors="black", alpha=0.3)
    plt.show()

In [ ]:
torch.manual_seed(42)
a = torch.distributions.gamma.Gamma(1,1/0.01)
s = a.sample((100,))
s_run4 = []
seed_ = seeds[0]
ret = LogReg().run(num_clients=5,
            data_set="synthetic_4",
            augment=True,#w^Tx+b if true, w^Tx otherwise
            train_test_split = 0.0,
            contamination={"contaminate": True, "type": "adversarial1", "epsilon": 0.05},
            heterogeneity=False,
            server_iterations=20,
            optim_epochs=500,
            lr = 0.005,
            monte_carlo_samples=100,
            minibatch_size = np.inf,
            client_batch_frac = 1.0,
            diff_prior_loc = None,
            diff_prior_cov = torch.tensor([1/s.mean()]),
            client_div = "KLD",
            client_div_param = None,
            client_loss = "gen_CE",
            client_score_fct = None,
            client_loss_param = 0.7,
            seed=seed_,
            return_data=True)
s_run4.append(ret)

In [ ]:
x_ = np.linspace(-3, 7, 500)
y_ = np.linspace(-3, 7, 500)

X, Y = np.meshgrid(x_,y_)

tx = s_run4[0]["train_data"]["x"]
ty = s_run4[0]["train_data"]["y"]
x_vals = tx[:, 1]
y_vals = tx[:, 2]
labels = ty

mu_s = s_run4[0]["q"]["loc"]
cov_s = s_run4[0]["q"]["var"]
num_samps = 1000
thetas = MultivariateNormal(loc=mu_s, covariance_matrix=torch.diag(cov_s)).sample((num_samps,)).numpy()
Probs4 = np.zeros_like(X)

for theta in thetas:
    """trial1 = -(theta[1]*X + theta[2]*Y + theta[0])
    trial2 = theta[0] + trial1
    trial3 = -trial2
    trial4 = np.exp(trial3)
    trial5 = 1 + trial4
    trial6 = 1 / trial5"""
    Probs4 += 1/(1+ np.exp(-(theta[0] + theta[1]*X + theta[2]*Y)))
    #Probs4 = Probs4 + trial6
Probs4 = Probs4/num_samps

with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.5), dpi = 200)
    plt.grid(visible=True)
    ax.scatter(x_vals[:100], y_vals[:100], c=labels[:100])
    ax.scatter(x_vals[:100], y_vals[:100], c=labels[:100])
    ax.scatter(x_vals[100:], y_vals[100:], c=labels[100:], marker='x')
    ctr0 = ax.contour(X,Y,Probs0, levels=[0.2,0.5,0.8], colors="black", alpha=.70, linestyles='dashed')
    ctr1 = ax.contour(X,Y,Probs, levels=[0.2,0.5,0.8], colors=cs[4])
    ctr2 = ax.contour(X,Y,Probs4, levels=[0.2,0.5,0.8], colors=cs[2])
    h0, _ = ctr0.legend_elements()
    h1, _ = ctr1.legend_elements()
    h2, _ = ctr2.legend_elements()
    ax.legend([h2[0],h1[0],h0[0]],[r'FedGVI','PVI', 'PVI no outliers'],loc="upper left", frameon=True)
    plt.show()

#### Plot for Paper

Chosen: Density Power Divergence based Loss $L_{GB}^{(0.7)}$ and Alpha- R\'enyi divergence $D_{AR}^{(1.5)}$

In [ ]:
x_ = np.linspace(-3, 7, 500)
y_ = np.linspace(-3, 7, 500)

X, Y = np.meshgrid(x_,y_)

tx = s_run2[0]["train_data"]["x"]
ty = s_run2[0]["train_data"]["y"]
x_vals = tx[:, 1]
y_vals = tx[:, 2]
labels = ty.type(torch.int64)

label_colors = np.array([cs[4], cs[0]]) 

cmap, norm = matplotlib.colors.from_levels_and_colors(range(0,3), label_colors)

labels_train = np.array(labels[:100])

labels_out = np.array(labels[100:])

with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 1.75), dpi = 200)
    plt.grid(visible=True, alpha=0.8)
    sc1 = ax.scatter(x_vals[:100], y_vals[:100], c=labels[:100])
    ctr0 = ax.contour(X,Y,Probs0, levels=[0.2,0.5,0.8], colors="black", alpha=.75, linestyles=[(0,(2.25,1.5))])
    ctr1 = ax.contour(X,Y,Probs, levels=[0.2,0.5,0.8], colors=cs[1])#, linestyles=[(0,(2.25,1.5))])
    sc2 = ax.scatter(x_vals[100:], y_vals[100:], c=labels[100:],cmap=cmap, marker='o')
    ctr2 = ax.contour(X,Y,Probs2, levels=[0.2,0.5,0.8], colors=cs[2])
    h0, _ = ctr0.legend_elements()
    h1, _ = ctr1.legend_elements()
    h2, _ = ctr2.legend_elements()
    h3, _ = sc2.legend_elements()
    h4, _ = sc1.legend_elements()
    ax.legend([h2[0],h1[0],h0[0],h4[0],h4[1],h3[0]],[r'FedGVI','PVI', 'PVI w/o Outliers','Class 0','Class 1', 'Outliers Class 0'],
              handlelength=2,loc="lower right", frameon=True, borderpad=0.4, fancybox=False, edgecolor=(210/255,210/255,210/255),
              framealpha=0.7)
    #fig.savefig('./figs/synthetic_log_reg_GB.pgf', format="pgf", bbox_inches= "tight", pad_inches=0.0)
    #fig.savefig('./figs/synthetic_log_reg_GB.pdf', format="pdf", bbox_inches= "tight")
    plt.show()

## Competing methods for Logistic Regression on the Covertype data set

In [ ]:
competing_results = Methods().run_competing_methods()

In [ ]:
minDSVGD, maxDSVGD, minFedAvg, maxFedAvg, minDSGLD, maxDSGLD = [],[],[],[],[],[]
avgDSVGD, avgFedAvg, avgDSGLD = [], [], []
iters = []
for i in range(10):
    iters.append(i+1)
    val_DSVGD_i = []
    val_FedAvg_i = []
    val_DSGLD_i = []
    for j in range(10):
        val_DSVGD_i.append(competing_results["DSVGD"]["validation"][j][i])
        val_FedAvg_i.append(competing_results["FedAvg"]["validation"][j][i])
        val_DSGLD_i.append(competing_results["DSGLD"]["validation"][j][i])
    minDSVGD.append(min(val_DSVGD_i))
    maxDSVGD.append(max(val_DSVGD_i))
    minFedAvg.append(min(val_FedAvg_i))
    maxFedAvg.append(max(val_FedAvg_i))
    minDSGLD.append(min(val_DSGLD_i))
    maxDSGLD.append(max(val_DSGLD_i))
    avgDSVGD.append(fmean(val_DSVGD_i))
    avgFedAvg.append(fmean(val_FedAvg_i))
    avgDSGLD.append(fmean(val_DSGLD_i))

In [ ]:
with plt.rc_context(bundles.icml2024()):
    fig, ax = plt.subplots(figsize = (3.25, 2.25), dpi = 200)
    
    '''ax.plot(iters,avg6, marker='s', label=r"VI")
    ax.fill_between(iters, min6, max6, alpha=0.3)
    ax.plot(iters,avg7, marker='p', label=r"GVI, $D_{AR}^{(5)}$")
    ax.fill_between(iters, min7, max7, alpha=0.3)
    ax.plot(iters,avg8, marker='v', label=r"GVI, $\mathcal{L}_{GCE}^{(0.5)}$")
    ax.fill_between(iters, min8, max8, alpha=0.3)'''


    ax.plot(iters,avg3, marker='D', label=r"PVI", color=cs2[0])
    ax.fill_between(iters, min3, max3, alpha=0.3, facecolor=cs2[0])

    ax.plot(iters,avgFedAvg, marker='*', label=r"FedAvg", color=cs2[1])
    ax.fill_between(iters, minFedAvg, maxFedAvg, alpha=0.3, facecolor=cs2[1])
    
    ax.plot(iters,avgDSGLD, marker='2', label=r"DSGLD", color=cs2[2])
    ax.fill_between(iters, minDSGLD, maxDSGLD, alpha=0.3, facecolor=cs2[2])
    
    ax.plot(iters,avgDSVGD, marker='P', label=r"DSVGD", color=cs2[4])
    ax.fill_between(iters, minDSVGD, maxDSVGD, alpha=0.3, facecolor=cs2[4])
    
    ax.plot(iters,avg4, marker='o', label=r"FedGVI, $D_{AR}^{(5)}$", color=cs2[5])
    ax.fill_between(iters, min4, max4, alpha=0.3, facecolor=cs2[5])
    ax.plot(iters,avg5, marker='^', label=r"FedGVI, $\mathcal{L}_{GCE}^{(0.5)}$", color=cs2[3])
    ax.fill_between(iters, min5, max5, alpha=0.3, facecolor=cs2[3])

    handles, labels = plt.gca().get_legend_handles_labels()
    order = [0,4,5,1,2,3]
    
    ax.set_xlabel(r"Server Iterations $t$")
    ax.set_ylabel(r"Classification Accuracy")
    ax.legend([handles[i] for i in order], [labels[i] for i in order],loc="lower right", frameon=True, ncol=2)
    ax.set_ylim([0.64,0.775])
    plt.grid(visible=True)
    fig.savefig('./figs/log_reg_fedgvi_competing.pgf', format="pgf", bbox_inches= "tight", pad_inches=0)
    plt.show()